In [1]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

# We are also going to do some basic viz
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Postgres info to connect

connection_args = {
  "host": "twitchdata.chd4n5ul8muk.us-east-2.rds.amazonaws.com",
  "user": "postgres",
    "password":"FwwBFmleh65qYxKxDVb9",
  "port": 5432,
  "dbname": "twitchdata"
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

In [4]:
query = "SELECT COUNT(DISTINCT(stream_id)) FROM stream_data;"

pd_sql.read_sql(query, connection)

,count
0,202178


In [5]:
query = "SELECT * FROM stream_data LIMIT 3;"

pd_sql.read_sql(query, connection)

,stream_id,user_id,user_name,game_id,stream_type,title,viewer_count,started_at,language,time_logged
0,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24
1,34368684464,273702366,Soddrek,19619,live,HOJE TEM GAZ AS 20H (18+) !burro !loots !lutab...,18,2019-06-01 22:48:35,pt,2019-06-01 23:03:24
2,34365706032,117766148,kyllini,19619,live,PxG Emerald - Kyllini - #OldTimesClan,16,2019-06-01 18:48:01,pt,2019-06-01 23:03:24


In [6]:
query = "SELECT COUNT(DISTINCT(game_id)) FROM game_information;"

pd_sql.read_sql(query, connection)

,count
0,465


In [12]:
query = "SELECT COUNT(DISTINCT(game_name)) FROM game_genres;"

pd_sql.read_sql(query, connection)

,count
0,1001


In [13]:
query = '''SELECT stream_id, user_id, user_name, stream_data.game_id, stream_type, title, viewer_count, started_at, language, time_logged, game_information.game_name, pic_url, game_genres.game_genres 
FROM stream_data 
LEFT JOIN game_information ON stream_data.game_id = game_information.game_id
LEFT JOIN game_genres ON game_information.game_name = game_genres.game_name;'''

data = pd_sql.read_sql(query, connection)

In [14]:
data.head(3)

,stream_id,user_id,user_name,game_id,stream_type,title,viewer_count,started_at,language,time_logged,game_name,pic_url,game_genres
0,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,RPG
1,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,Adventure Game
2,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,MMORPG


In [15]:
data.to_csv('tuesday_merged_data.csv')

In [3]:
data = pd.read_csv('tuesday_merged_data.csv')

In [17]:
data.columns

Index(['stream_id', 'user_id', 'user_name', 'game_id', 'stream_type', 'title',
       'viewer_count', 'started_at', 'language', 'time_logged', 'game_name',
       'pic_url', 'game_genres'],
      dtype='object')

In [18]:
data = data.dropna()

In [19]:
data.shape

(1719321, 13)

In [20]:
data['game_id'].nunique()

331

In [21]:
data['user_id'].nunique()

122043

In [22]:
data['game_name'].nunique()

331

In [23]:
data['user_name'].nunique()

122220

In [24]:
data.head(3)

,stream_id,user_id,user_name,game_id,stream_type,title,viewer_count,started_at,language,time_logged,game_name,pic_url,game_genres
0,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,RPG
1,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,Adventure Game
2,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,MMORPG


In [9]:
del data
import gc
gc.collect()

72

In [30]:
#data.pivot_table(index='user_id', columns='game_id', values='user_id', aggfunc=len, fill_value=0)

In [5]:
grid = pd.crosstab(data.user_id, data.game_id)

In [11]:
grid = grid.astype('float16')

In [12]:
grid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122605 entries, 6410 to 439730780
Columns: 465 entries, 104 to 512812
dtypes: float16(465)
memory usage: 109.7 MB


In [28]:
grid.to_csv('grid.csv')

In [27]:
names = pd.crosstab(data.user_name, data.game_name)

In [29]:
names.to_csv('names.csv')

In [13]:
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

In [ ]:
from scipy.linalg import svd

U, Sigma, VT = svd(grid)

In [ ]:
VT = VT[:3,:]
pd.DataFrame(VT)